In [1]:
from pImpactR import MLI as mli
import pandas as pd
import numpy as np

# Map

### global settings

In [2]:
beam = mli.getElem.beam(name='beam',particle='electron',energy=0.150511006)
units = mli.getElem.units(name='units',type='static')
setting = mli.getElem.globaldefaults(name='setdefaults',lfrngsbend=1, tfrngsbend=1, lfrngquad=1, tfrngquad=1)

beamStr ='#menu \n' \
         +beam.str() + ' \n' \
         +units.str() + ' \n' \
         +setting.str() + ' \n'

### lattice

In [3]:
with open('lattice.mli') as f:
    latticeStr = f.read() + '\n'

### commands and labor

In [4]:
mapout = mli.getElem.ptm(name='mapout',order=3)
labor = mli.labor(['iotaline',mapout,mli.getElem.fin])
cmdStr = mapout.str() + ' \n'

In [5]:
with open('mli.in','w') as f:
    f.write(beamStr + latticeStr + cmdStr + labor.str())

### results

In [6]:
mli.run()

In [7]:
M=mli.readMatrix()
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [8]:
G=mli.readGeneratingPolynomial()
G[:83]

,exponents,value
28,f( 30 00 00 ),7.765582e-12
29,f( 21 00 00 ),-5.701697e-12
33,f( 20 00 01 ),7.272738e-01
34,f( 12 00 00 ),-2.987872e-12
38,f( 11 00 01 ),5.580342e-12
39,f( 10 20 00 ),-7.310090e-12
40,f( 10 11 00 ),-2.825363e-12
43,f( 10 02 00 ),1.654872e-12
48,f( 10 00 02 ),3.322873e-12
49,f( 03 00 00 ),-6.347985e-11


# Tracking

### prepare particle data

In [9]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx

Espread = 1.0e-3
bet0=0.999942
npt = 64

In [10]:
def MLI2norm(data_in,bet0=1.0,sign=1):
    data=data_in.copy()
    data[:,5] = -data[:,5]*bet0
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2MLI(data_in,bet0=1.0,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = -data[:,5]/bet0
    return data

In [11]:
x = (np.random.random((npt-1)*5)-0.5)/0.5*0.01
pData0 = np.zeros([64,6])
pData1 = np.zeros([64,6])
pData1[0,:] = [8.797241825320807271e-05,
               6.665913855307434126e-05,
               -7.571314763642334855e-05,
               1.103354218171259671e-04,
               0.000000000000000000e+00,
               -6.947507409701951520e-03]
pData1[1:,[0,1,2,3,5]]=x.reshape([npt-1,5])
pData0[:,:4] = pData1[:,:4]

### commands

In [12]:
dump=mli.getElem.particledump(name='dump',fname='rays.out')
iota_tbt=mli.getElem.line(name='iota_tbt',elemList=['iotaline',dump])
read=mli.getElem.raytrace(name='read',fname='rays.in')
track=mli.getElem.autotrack(name='track',type='symplectic',order=5)

### non-zero energy spread, symplecticN

In [13]:
np.savetxt('rays.in',pData1)
results = []
for n in range(1,4):
    track.order = n
    cmdStr = mli.cmd2str([dump,iota_tbt,read,track])
    labor = mli.labor([read,track,1,iota_tbt,mli.getElem.fin])
    with open('mli.in','w') as f:
        f.write(beamStr + latticeStr + cmdStr + labor.str())
    mli.run()    
    pDataOut = mli.getTBT(npt,1)[:,0,:]
    pDataOut = MLI2norm(pDataOut,bet0=0.999942)
    results.append(pDataOut)
    
refData = MLI2norm(pData1,bet0=0.999942,sign=-1)

outputs = []
for i in range(npt):
    outputs.append(pd.DataFrame(columns=['x','px','y','py','delta'], index=['original','symplectic1','symplectic2','symplectic3']))
    outputs[i].iloc[0]=refData[i,[0,1,2,3,5]]
    outputs[i].iloc[1]=results[0][i,[0,1,2,3,5]]
    outputs[i].iloc[2]=results[1][i,[0,1,2,3,5]]
    outputs[i].iloc[3]=results[2][i,[0,1,2,3,5]]
    
outputs[0]

,x,px,y,py,delta
original,0.0063946,0.000305813,-0.00550349,0.0226494,0.0069471
symplectic1,0.00639532,0.00042807,-0.00550346,0.0225447,0.0069471
symplectic2,0.258514,0.00936363,9.89278e-05,0.0253419,0.0069471
symplectic3,0.229678,-0.00303169,-0.00185463,0.0255903,0.0069471


### zero energy spread, symplecticN

In [14]:
np.savetxt('rays.in',pData0)
results = []
for n in range(1,4):
    track.order = n
    cmdStr = mli.cmd2str([dump,iota_tbt,read,track])
    labor = mli.labor([read,track,1,iota_tbt,mli.getElem.fin])
    with open('mli.in','w') as f:
        f.write(beamStr + latticeStr + cmdStr + labor.str())
    mli.run()    
    pDataOut = mli.getTBT(npt,1)[:,0,:]
    pDataOut = MLI2norm(pDataOut,bet0=0.999942)
    results.append(pDataOut)
    
refData = MLI2norm(pData0,bet0=0.999942,sign=-1)

outputs = []
for i in range(npt):
    outputs.append(pd.DataFrame(columns=['x','px','y','py','delta'], index=['original','symplectic1','symplectic2','symplectic3']))
    outputs[i].iloc[0]=refData[i,[0,1,2,3,5]]
    outputs[i].iloc[1]=results[0][i,[0,1,2,3,5]]
    outputs[i].iloc[2]=results[1][i,[0,1,2,3,5]]
    outputs[i].iloc[3]=results[2][i,[0,1,2,3,5]]
    
outputs[0]

,x,px,y,py,delta
original,0.0063946,0.000369082,-0.00550349,0.0227541,-0
symplectic1,0.00639474,0.000368847,-0.00550346,0.0227539,-0
symplectic2,0.00640021,0.000361785,-0.00551362,0.0227579,-0
symplectic3,0.0064,0.000361599,-0.00551634,0.0227579,-0


### non-zero energy spread, taylorN

In [15]:
np.savetxt('rays.in',pData1)
track.type = 'taylor'
results = []
for n in range(2,5):
    track.order = n
    cmdStr = mli.cmd2str([dump,iota_tbt,read,track])
    labor = mli.labor([read,track,1,iota_tbt,mli.getElem.fin])
    with open('mli.in','w') as f:
        f.write(beamStr + latticeStr + cmdStr + labor.str())
    mli.run()    
    pDataOut = mli.getTBT(npt,1)[:,0,:]
    pDataOut = MLI2norm(pDataOut,bet0=0.999942)
    results.append(pDataOut)
    
refData = MLI2norm(pData1,bet0=0.999942,sign=-1)

outputs = []
for i in range(npt):
    outputs.append(pd.DataFrame(columns=['x','px','y','py','delta'], index=['original','taylor2','taylor3','taylor4']))
    outputs[i].iloc[0]=refData[i,[0,1,2,3,5]]
    outputs[i].iloc[1]=results[0][i,[0,1,2,3,5]]
    outputs[i].iloc[2]=results[1][i,[0,1,2,3,5]]
    outputs[i].iloc[3]=results[2][i,[0,1,2,3,5]]
    
outputs[0]

,x,px,y,py,delta
original,0.0063946,0.000305813,-0.00550349,0.0226494,0.0069471
taylor2,0.219544,-0.0063208,-0.00251236,0.0253691,0.0069471
taylor3,0.229391,-0.00335349,-0.00187596,0.0256015,0.0069471
taylor4,0.229207,-0.00341164,-0.00188782,0.0256003,0.0069471


### zero energy spread, taylorN

In [16]:
np.savetxt('rays.in',pData0)
results = []
for n in range(2,5):
    track.order = n
    cmdStr = mli.cmd2str([dump,iota_tbt,read,track])
    labor = mli.labor([read,track,1,iota_tbt,mli.getElem.fin])
    with open('mli.in','w') as f:
        f.write(beamStr + latticeStr + cmdStr + labor.str())
    mli.run()    
    pDataOut = mli.getTBT(npt,1)[:,0,:]
    pDataOut = MLI2norm(pDataOut,bet0=0.999942)
    results.append(pDataOut)
    
refData = MLI2norm(pData0,bet0=0.999942,sign=-1)

outputs = []
for i in range(npt):
    outputs.append(pd.DataFrame(columns=['x','px','y','py','delta'], index=['original','taylor2','taylor3','taylor4']))
    outputs[i].iloc[0]=refData[i,[0,1,2,3,5]]
    outputs[i].iloc[1]=results[0][i,[0,1,2,3,5]]
    outputs[i].iloc[2]=results[1][i,[0,1,2,3,5]]
    outputs[i].iloc[3]=results[2][i,[0,1,2,3,5]]
    
outputs[0]

,x,px,y,py,delta
original,0.0063946,0.000369082,-0.00550349,0.0227541,-0
taylor2,0.00640013,0.000362018,-0.00551602,0.0227577,-0
taylor3,0.0064,0.000361599,-0.00551634,0.0227579,-0
taylor4,0.0064,0.000361599,-0.00551634,0.0227579,-0
